Copyright 2021 D. Bazzett  
The purpose of this code is to look at the COMIDs associated with the HWM data and create a custom hydrogeo-table (rating curve) for only these COMIDs. 99% of the data in the default tables is not included in the HWM data, so creating a custom table speeds up calculation times.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# for comid in ['020301', '020401', '020402', '020403']:
comid = '020301'
hydrotable_file = "H:/My Drive/" + comid + "/hydrogeo-fulltable-" + comid + ".csv"
df1 = pd.read_csv(hydrotable_file)
print(len(df1.index))

for comid in ['020401', '020402', '020403']:
    hydrotable_file = "H:/My Drive/" + comid + "/hydrogeo-fulltable-" + comid + ".csv"
    df2 = pd.read_csv(hydrotable_file)
    df1 = df1.append(df2)
    print(len(df2.index))
df1

427035
609386
1006624
742020


,CatchId,Stage,Number of Cells,SurfaceArea (m2),BedArea (m2),Volume (m3),SLOPE,LENGTHKM,AREASQKM,Roughness,TopWidth (m),WettedPerimeter (m),WetArea (m2),HydraulicRadius (m),Discharge (m3s-1)
0,6227452,0.0000,133,10604.098582,10717.675041,0.000000,0.123987,1.264,1.2339,0.05,8.389318,8.479173,0.000000,0.000000,0.000000
1,6227452,0.3048,149,11879.771715,11993.653778,3422.739011,0.123987,1.264,1.2339,0.05,9.398554,9.488650,2.707863,0.285379,8.265965
2,6227452,0.6096,166,13235.164248,13350.933168,7253.803373,0.123987,1.264,1.2339,0.05,10.470858,10.562447,5.738768,0.543318,26.909573
3,6227452,0.9144,193,15387.841052,15510.719239,11599.104095,0.123987,1.264,1.2339,0.05,12.173925,12.271139,9.176506,0.747812,53.242411
4,6227452,1.2192,217,17301.314961,17434.150253,16503.575210,0.123987,1.264,1.2339,0.05,13.687749,13.792840,13.056626,0.946623,88.647766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742015,10466721,23.7744,412,35013.261198,35044.433869,817772.424058,0.000010,0.125,0.0351,0.05,280.106090,280.355471,6542.179392,23.335301,3378.791450
742016,10466721,24.0792,412,35013.261198,35044.433869,828444.466071,0.000010,0.125,0.0351,0.05,280.106090,280.355471,6627.555729,23.639830,3452.600155
742017,10466721,24.3840,412,35013.261198,35044.433869,839116.508084,0.000010,0.125,0.0351,0.05,280.106090,280.355471,6712.932065,23.944359,3527.045473
742018,10466721,24.6888,412,35013.261198,35044.433869,849788.550097,0.000010,0.125,0.0351,0.05,280.106090,280.355471,6798.308401,24.248888,3602.124691


In [3]:
# hwms = pd.read_csv('./Cleaned Hwm/HWM_Combined_NoNegatives_Cleaned.csv')
# hwms = pd.read_csv('H:/My Drive/CODE/HAND/Cleaned Hwm/RAW/USGS_Somerset_DEM_HAND_HUC_calc_no-neg_clean.csv')
# hwms = pd.read_csv('H:/My Drive/CODE/HAND/HWM/USGS_Somerset_opt.csv')
hwms = pd.read_csv('H:/My Drive/CODE/HAND/HWM/USGS_Somerset_opt.csv')
hwms

,Unnamed: 0,Unnamed: 0.1,SITE_NUM,HWM_ID,SITE_ID,NAME,LAT_DD,LON_DD,ELEV_NAVD88_ft,ELEV_NAVD88_m,...,COMID,Inun_m,Stream_H_m,Cleaning_note,Error_note,fmax,time_file,roughness,rmse,Predict_Stream_H_m
0,0,0,132.0,1.0,SADD9,SADD9_1,41.075500,-74.084528,273.4055,83.333996,...,NaN,NaN,NaN,deleted,"Flowline/HAND are incorrect, depth should be less",0.0,0,NaN,NaN,NaN
1,1,1,132.0,2.0,SADD9,SADD9_2,41.076000,-74.085417,270.8655,82.559804,...,NaN,NaN,NaN,deleted,"Flowline/HAND are incorrect, depth should be less",0.0,0,NaN,NaN,NaN
2,2,2,132.0,3.0,SADD9,SADD9_3,41.075944,-74.085250,270.0155,82.300724,...,NaN,NaN,NaN,deleted,"Flowline/HAND are incorrect, depth should be less",0.0,0,NaN,NaN,NaN
3,3,3,132.0,4.0,SADD9,SADD9_4,41.075861,-74.085028,272.9355,83.190740,...,NaN,NaN,NaN,deleted,"Flowline/HAND are incorrect, depth should be less",0.0,0,NaN,NaN,NaN
4,4,4,132.0,5.0,SADD9,SADD9_5,41.075389,-74.085167,266.3655,81.188204,...,NaN,NaN,NaN,deleted,"Flowline/HAND are incorrect, depth should be less",0.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037,1037,1037,138.0,5.0,SBRA1,SBRA1_5,40.510000,-74.726361,NaN,0.000000,...,NaN,-21.479713,NaN,deleted,NaN,0.0,0,NaN,NaN,NaN
1038,1038,1038,140.0,3.0,SBRA4,SBRA4_3,40.636472,-74.911611,195.6975,59.648598,...,NaN,1.687149,NaN,deleted,NaN,0.0,0,NaN,NaN,NaN
1039,1039,1039,157.0,6.0,WHIP10,WHIP10_6,40.793583,-74.567778,NaN,0.000000,...,NaN,-128.523926,NaN,deleted,NaN,0.0,0,NaN,NaN,NaN
1040,1040,1040,154.0,1.0,WHIP7,WHIP7_1,40.812500,-74.480944,313.4775,95.547942,...,NaN,-0.997873,NaN,deleted,NaN,0.0,0,NaN,NaN,NaN


In [4]:
# for comid in hwms.COMID.unique():
#     subset = hwms[hwms.COMID == comid]
#     print(subset.group_avg.mean())
#     if not np.isnan(subset.group_avg.mean()):
#         print('completed')
#         continue
#     print(comid)

In [5]:
# maxflow = pd.read_csv('H:/My Drive/CODE/HAND/NWM Flow/maxflow_timefile.csv')
# hwms = hwms.merge(maxflow, on='COMID', how='left')
# hwms.to_csv('H:/My Drive/CODE/HAND/Cleaned Hwm/RAW/USGS_Somerset_opt.csv')

In [6]:
len(hwms.COMID.unique())

191

In [7]:
# df1.loc[df1.CatchId == 6251308, 'SLOPE'] = 0.05
# df1[df1.CatchId == 6251308]

In [8]:
# 020301
df1.loc[df1.CatchId == 6251308, 'SLOPE'] = 0.15 / 5223 # PASS1
df1.loc[df1.CatchId == 6251234, 'SLOPE'] = 0.36 / 6934 # PASS2 
df1.loc[df1.CatchId == 6251128, 'SLOPE'] = 1.27 / 1729 # PASS3
df1.loc[df1.CatchId == 6249784, 'SLOPE'] = 0.00005 # PASS7 - arbitrary because slope is almost zero
df1.loc[df1.CatchId == 6251122, 'SLOPE'] = 0.37 / (694 + 701) # PASS10
df1.loc[df1.CatchId == 6251144, 'SLOPE'] = 0.37 / (694 + 701) # PASS10
df1.loc[df1.CatchId == 6250358, 'SLOPE'] = 0.16 / 2744 # PASS16
df1.loc[df1.CatchId == 6250388, 'SLOPE'] = 0.71 / 3277 # PASS17
df1.loc[df1.CatchId == 6250418, 'SLOPE'] = 1.40 / 2824 # PASS18

df1.loc[df1.CatchId == 6246206, 'SLOPE'] = 1.59 / 2824 # WANQ2
df1.loc[df1.CatchId == 6250622, 'SLOPE'] = 1.41 / 2480 # POMP4/5
df1.loc[df1.CatchId == 6249738, 'SLOPE'] = 0.00005     # POMP2 - arbitrary value to remove error
# df1[df1.CatchId == 6246098, 'SLOPE'] = 0.88 / 1013 # RAMA5 - 
df1.loc[df1.CatchId == 932030072, 'SLOPE']=1.75 / 635  # RAMA5
df1.loc[df1.CatchId == 6250680, 'SLOPE'] = 0.00005     # ROCK2 - arbitrary value to remove error

# 020402
df1.loc[df1.CatchId == 4491888, 'SLOPE'] = 0.5 / 3571 # NBRAN4

df1.loc[df1.SLOPE < 0.00003, 'SLOPE'] = 0.00003

df1['Discharge (m3s-1)'] = df1['WetArea (m2)'] * pow(df1['HydraulicRadius (m)'],2.0/3) * \
    pow(df1['SLOPE'],0.5) / df1['Roughness']

In [9]:
# df1[df1.CatchId == 4491888]

In [15]:
hwms = hwms.dropna(subset=['COMID'])
valid_rows1 = df1[df1['CatchId'].isin(set(hwms.COMID.unique()))]  
valid_rows1

,CatchId,Stage,Number of Cells,SurfaceArea (m2),BedArea (m2),Volume (m3),SLOPE,LENGTHKM,AREASQKM,Roughness,TopWidth (m),WettedPerimeter (m),WetArea (m2),HydraulicRadius (m),Discharge (m3s-1)
94371,6249688,0.0000,1611,1.291823e+05,1.291853e+05,0.000000e+00,0.000612,5.019,8.6067,0.05,25.738648,25.739246,0.000000,0.000000,0.000000
94372,6249688,0.3048,2065,1.655891e+05,1.655974e+05,4.484286e+04,0.000612,5.019,8.6067,0.05,32.992439,32.994103,8.934620,0.270794,1.849799
94373,6249688,0.6096,2400,1.924527e+05,1.924910e+05,9.923374e+04,0.000612,5.019,8.6067,0.05,38.344837,38.352455,19.771615,0.515524,6.287771
94374,6249688,0.9144,3410,2.734475e+05,2.735614e+05,1.678245e+05,0.000612,5.019,8.6067,0.05,54.482473,54.505170,33.437828,0.613480,11.941537
94375,6249688,1.2192,4409,3.535589e+05,3.538344e+05,2.639289e+05,0.000612,5.019,8.6067,0.05,70.444094,70.498981,52.585955,0.745911,21.393555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157197,9441129,23.7744,64562,5.233016e+06,5.244437e+06,6.168122e+07,0.001767,2.598,5.6457,0.05,2014.247880,2018.643799,23741.809992,11.761268,103220.564563
157198,9441129,24.0792,65024,5.270464e+06,5.282043e+06,6.328239e+07,0.001767,2.598,5.6457,0.05,2028.661892,2033.118988,24358.116210,11.980664,107212.959040
157199,9441129,24.3840,65480,5.307425e+06,5.319170e+06,6.489448e+07,0.001767,2.598,5.6457,0.05,2042.888687,2047.409404,24978.630596,12.200115,111282.676652
157200,9441129,24.6888,65876,5.339523e+06,5.351407e+06,6.651735e+07,0.001767,2.598,5.6457,0.05,2055.243591,2059.817974,25603.291695,12.429881,115493.292135


In [11]:
# valid_rows1.to_csv('H:/My Drive/CODE/HAND/HWM/hydrogeo-fulltable-020999.csv')

In [13]:
gages = pd.read_csv('H:/My Drive/CODE/HAND/HWM/GagesNJ_Height_Flow_Cleaned.csv')
gages

,Unnamed: 0,Unnamed: 0.1,agency_cd,site_no,datetime,tz_cd,194264_00065,Stream_H_m,194264_00065_cd,OID_,...,FLComID,POINT_X,POINT_Y,POINT_Z,POINT_M,COMID,time_file,Predict_Stream_H_m,roughness,rmse
0,0,0,USGS,1389802,2011-08-30 03:00,EDT,122.54,37.350192,A,708,...,6251048,-74.181557,40.915657,0,79.85481,6251048,20110829_01,NaN,NaN,NaN
1,1,1,USGS,1403060,2011-08-28 18:00,EDT,41.90,12.771120,A,490,...,9513378,-74.548574,40.551601,0,68.90423,9513378,20110828_19,NaN,NaN,NaN
2,2,2,USGS,1400500,2011-08-28 16:00,EDT,26.29,8.013192,A,491,...,9513852,-74.583119,40.555555,0,89.83748,9513852,20110828_18,NaN,NaN,NaN
3,3,3,USGS,1388500,2011-08-29 01:15,EDT,25.24,7.693152,A,720,...,6249470,-74.281760,40.969298,0,56.37115,6249470,20110828_16,NaN,NaN,NaN
4,4,4,USGS,1446995,2011-08-29 05:00,EDT,25.15,7.665720,A,565,...,2588457,-75.203571,40.694496,0,86.53518,2588457,20110830_19,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,147,149,USGS,1409000,2011-08-28 09:45,EDT,4.32,1.316736,A,255,...,9453855,-74.169190,39.867962,0,18.62468,9453855,20110828_08,NaN,NaN,NaN
130,148,150,USGS,1396800,2011-08-28 14:30,EDT,4.06,1.237488,A,569,...,9513994,-74.915253,40.639259,0,0.00322,9512994,20110828_09,NaN,NaN,NaN
131,149,151,USGS,1379773,2011-08-28 19:45,EDT,3.84,1.170432,A,692,...,6249510,-74.539606,40.959445,0,13.19529,6249510,20110828_13,NaN,NaN,NaN
132,150,153,USGS,1443900,2011-08-28 10:30,EDT,3.22,0.981456,A,664,...,2587723,-75.039398,40.981147,0,62.34066,2587723,20110828_10,NaN,NaN,NaN


In [16]:
valid_rows2 = df1[df1['CatchId'].isin(set(gages.COMID.unique()))]  
valid_rows2

,CatchId,Stage,Number of Cells,SurfaceArea (m2),BedArea (m2),Volume (m3),SLOPE,LENGTHKM,AREASQKM,Roughness,TopWidth (m),WettedPerimeter (m),WetArea (m2),HydraulicRadius (m),Discharge (m3s-1)
112216,6251048,0.0000,734,58874.081036,58896.571596,0.000000e+00,0.022008,0.961,0.6426,0.05,61.263352,61.286755,0.000000,0.000000,0.000000
112217,6251048,0.3048,915,73391.794625,73416.029229,2.064217e+04,0.022008,0.961,0.6426,0.05,76.370234,76.395452,21.479886,0.281167,27.352587
112218,6251048,0.6096,1088,87268.042346,87301.360973,4.523489e+04,0.022008,0.961,0.6426,0.05,90.809617,90.844288,47.070646,0.518146,90.096647
112219,6251048,0.9144,1219,97775.489417,97823.474411,7.344203e+04,0.022008,0.961,0.6426,0.05,101.743485,101.793418,76.422507,0.750761,187.303450
112220,6251048,1.2192,1327,106438.189966,106515.212922,1.045021e+05,0.022008,0.961,0.6426,0.05,110.757742,110.837891,108.743115,0.981101,318.567078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445622,9436375,23.7744,2594,212584.430302,212837.362833,4.581029e+06,0.000598,0.301,0.2124,0.05,706.260566,707.100873,15219.366823,21.523615,57595.335591
445623,9436375,24.0792,2594,212584.430302,212837.362833,4.645825e+06,0.000598,0.301,0.2124,0.05,706.260566,707.100873,15434.635044,21.828053,58959.475783
445624,9436375,24.3840,2594,212584.430302,212837.362833,4.710621e+06,0.000598,0.301,0.2124,0.05,706.260566,707.100873,15649.903264,22.132490,60336.359259
445625,9436375,24.6888,2594,212584.430302,212837.362833,4.775417e+06,0.000598,0.301,0.2124,0.05,706.260566,707.100873,15865.171485,22.436928,61725.927316


In [17]:
valid_rows2.to_csv('H:/My Drive/CODE/HAND/HWM/hydrogeo-fulltable-020998.csv')